# Capstone Project - The Battle of Neighborhoods!

Install and import required packages

In [ ]:
# install the Google Trends API
!pip install pytrends

# install the Daft Listings API
!pip install daftlistings==1.8.1

In [9]:
# python packages
import pprint
import requests
import geopandas
import pyproj as pp
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Google Trends API packages
from pytrends.request import TrendReq

# Daft listings API packages
from daftlistings import Daft, RentType, SortOrder, SortType, MapVisualization
from joblib import Parallel, delayed
import time

# convert an address into latitude and longitude values
from geopy.geocoders import Nominatim

import folium # map rendering library

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

## 1. Introduction
This section outlines a general background for the Business Problem that I'll be trying to solve as part of the capstone project.

The primary focus for this project would be on the city Dublin and its 22 different District areas.  

This project tries to achieve the following analyses for the respective target audience in mind:  
1) **House Renting**: Finding an apartment to rent in Dublin city is very challenging given the housing crisis. The target audience in this case is people looking for rental apartments in the city. The attempt here is to filter out properties based on user preferences for apartment characteristics, neighborhood choices, pricing and crime rate in the neighborhood in which the property is situated.  
2) **Neighborhood Clustering**: The approach here is to use visualization techniques to cluster districts within Dublin city using clustering techniques based on the venues and venue categories present in different districts. We can get a sense of how different districts are oriented within the city in terms of different places, amenities, transport routes and most importantly whether distance from the city centre plays a role in driving this.  
3) **Google Trends**: This data would act as one of the features where we try to do regerssion analysis for predicting the rent price for each apartment. The hypothesis would be that google trends for a search for an apartment to rent in a particular neighborhood would affect the pricing for the rentals. The analysis performed in the subsequent report would test this hypothesis.  
4) **Crimes**: This data would act as additional filtering for users looking to rent an apartment as well as drive the clustering of the districts as planned in point 2 above. It would be intersting to use visualizatin techniques again to find out if crimes are related to the geograhphical attributes of a particular neighborhood.    

Overall the aim is to aid people looking for rentals in Dublin city and help them filter out neighborhoods and properties based on their preferences as well as other local factors driving their decision making.  
Apart from that, the visualiztion techniques used for analysing different datasets would help certain stakeholders make decisions in terms of government planning, business marketing decisions as well as general readers looking for some insights of their own city! 

## 2. Data
This section defines the different data sources as well as their sample examples that have been used for this assignment.

### 1) Google Trends API
Please find below sample of how we plan to use this API (https://pypi.org/project/pytrends/)  
As seen in the example below, we are basically getting month-wise interests for apartments searches containing different districts within Dublin city.  
The plan is to aggregate the data to engineer features for the pricing predective model that we plan to build in the sections to follow.  

In [132]:
pytrends = TrendReq(hl='en-US', tz=0)

kw_list = ["Dublin 1", "Dublin 2"]
pytrends.build_payload(kw_list, geo='IE')

pytrends.interest_over_time()

,Dublin 1,Dublin 2,isPartial
date,,,
2015-06-28,51,88,False
2015-07-05,52,88,False
2015-07-12,44,81,False
2015-07-19,55,76,False
2015-07-26,40,88,False
2015-08-02,64,83,False
2015-08-09,47,80,False
2015-08-16,52,78,False
2015-08-23,54,69,False


### 2) Daft Listings API
As seen below, this is a very useful API (https://github.com/AnthonyBloomer/daftlistings/) yet simple to use and get upto speed.  
The sample example below shows a search using the API to get all listings in "Dublin city for rental 3-bed apartments with a max price of 2800EUR and furnished".  
We fetch all such listings and build a dataframe containing all the useful features for each property which as seen below would consist of <price', 'facilities', 'formalised_address', 'num_bedrooms', 'num_bathrooms', 'latitude', 'longitude'>  
This data would help us recommend properties to the targeted end-user as well as the geographical  coordinates would help us visually analyse the data in question.  

In [2]:
def translate_listing_to_json(listing):
    try:
        if listing.search_type != 'rental':
            return None
        return listing.as_dict()
    except:
        return None

In [3]:
daft = Daft()
daft.set_county("Dublin City")
daft.set_listing_type(RentType.ANY)
daft.set_max_price(2800)
daft.set_min_beds(3)
daft.set_max_beds(3)
daft.set_furnished(True)
daft.set_sort_order(SortOrder.ASCENDING)
daft.set_sort_by(SortType.PRICE)

In [4]:
listings = daft.search()
properties = []
print("Translating {} listing object into json, it will take a few minutes".format(str(len(listings))))
print("Igonre the error message")

# time the translation
start = time.time()
properties = Parallel(n_jobs=6, prefer="threads")(delayed(translate_listing_to_json)(listing) for listing in listings)
properties = [p for p in properties if p is not None] # remove the None
end = time.time()
print("Time for json translations {}s".format(end-start))

http://www.daft.ie/dublin-city/residential-property-for-rent/?offset=0&s%5Bmnb%5D=3&s%5Bmxb%5D=3&s%5Bfurn%5D=1&s%5Bsort_type%5D=a&s%5Bsort_by%5D=price&s%5Bmxp%5D=2800&s%5Bignored_agents%5D%5B1%5D
Fetching page 1 of 16 
Fetching page 2 of 16 
Fetching page 3 of 16 
Fetching page 4 of 16 
Fetching page 5 of 16 
Fetching page 6 of 16 
Fetching page 7 of 16 
Fetching page 8 of 16 
Fetching page 9 of 16 
Fetching page 10 of 16 
Fetching page 11 of 16 
Fetching page 12 of 16 
Fetching page 13 of 16 
Fetching page 14 of 16 
Fetching page 15 of 16 
Fetching page 16 of 16 
Fetched 309 listings.
Translating 309 listing object into json, it will take a few minutes
Igonre the error message


ERROR:root:Error getting price_change. Error message: 'NoneType' object has no attribute 'text'
ERROR:root:Error getting price_change. Error message: 'NoneType' object has no attribute 'text'
ERROR:root:Error getting agent. Error message: 'NoneType' object has no attribute 'text'
ERROR:root:Error getting agent_url. Error message: 'NoneType' object has no attribute 'find_all'
ERROR:root:Error getting contact_number. Error message: 'NoneType' object has no attribute 'attrs'
ERROR:root:Error getting contact_number. Error message: 'NoneType' object has no attribute 'attrs'
ERROR:root:Error getting commercial_area_size. Error message: 'NoneType' object has no attribute 'find'
ERROR:root:Error getting commercial_area_size. Error message: 'NoneType' object has no attribute 'find'
ERROR:root:Error getting price_change. Error message: 'NoneType' object has no attribute 'text'
ERROR:root:Error getting price_change. Error message: 'NoneType' object has no attribute 'text'
ERROR:root:Error getting

Time for json translations 99.97307896614075s


In [5]:
listings[:5]

[Listing (St. Edmunds, St Edmunds, Lucan, Co. Dublin),
 Listing (Island Key, East Road, Dublin 3),
 Listing (U Citywest Demesne, Citywest, Citywest, Co. Dublin),
 Listing (Clancy Quay By Kennedy Wilson, South Circular Road, Dublin 8),
 Listing (From Here)]

In [6]:
df = pd.DataFrame(properties)
df = df[['price', 'facilities', 'formalised_address', 'num_bedrooms', 'num_bathrooms', 'latitude', 'longitude']]
df.head()

,price,facilities,formalised_address,num_bedrooms,num_bathrooms,latitude,longitude
0,1265,[],"Grange View Lawn, Clondalkin, Clondalkin, Dubl...",3,1,53.323175,-6.428534
1,1592,"[Parking, Central Heating, Cable Television, W...","Kilcarrig, Tallaght, Dublin 24, South Co. Dublin",3,1,53.29276631413049,-6.392316040899743
2,1600,"[Parking, Central Heating, House Alarm, Cable ...","Decies Road, Ballyfermot, Dublin 10, South Dub...",3,1,53.340353,-6.339297
3,1650,[],"Boroimhe Beech, Swords, North Co. Dublin",3,2,53.443901,-6.233472
4,1695,"[Parking, Central Heating, Cable Television, W...","The Wood, Millbrook Lawns, Tallaght, Dublin 24...",3,1,53.281367513416114,-6.356052283775597


### 3) Ireland Crimes CSV
This dataset has been obtained from https://data.gov.ie/dataset/crimes-at-garda-stations-level-2010-2016 which gives a statistical breakdown of different types of crimes committed within the areas that come under each of the different police (garda) stations in Ireland.  
Sample below shows the dataset specifics, where we can see the different year-wise columns representing different types of crimes as well as x and y geographical coordinates would help us do some high-level visual analysis.  
As said earlier, this dataset would be aggregated for each of the districts in Dublin and could help us effectively filter out districts having larger crime rates.  

In [2]:
crimes_df = pd.read_csv('crimes_garda_stations.csv', encoding = "ISO-8859-1")
crimes_df.head()

,id,Station,Divisions,x,y,"Attempts or threats to murder, assaults, harassments and related offences 2004","Attempts or threats to murder, assaults, harassments and related offences 2005","Attempts or threats to murder, assaults, harassments and related offences 2006","Attempts or threats to murder, assaults, harassments and related offences 2007","Attempts or threats to murder, assaults, harassments and related offences 2008","Attempts or threats to murder, assaults, harassments and related offences 2009","Attempts or threats to murder, assaults, harassments and related offences 2010","Attempts or threats to murder, assaults, harassments and related offences 2011","Attempts or threats to murder, assaults, harassments and related offences 2012","Attempts or threats to murder, assaults, harassments and related offences 2013","Attempts or threats to murder, assaults, harassments and related offences 2014","Attempts or threats to murder, assaults, harassments and related offences 2015","Attempts or threats to murder, assaults, harassments and related offences 2016*",Dangerous or negligent acts 2003,Dangerous or negligent acts 2004,Dangerous or negligent acts 2005,Dangerous or negligent acts 2006,Dangerous or negligent acts 2007,Dangerous or negligent acts 2008,Dangerous or negligent acts 2009,Dangerous or negligent acts 2010,Dangerous or negligent acts 2011,Dangerous or negligent acts 2012,Dangerous or negligent acts 2013,Dangerous or negligent acts 2014,Dangerous or negligent acts 2015,Dangerous or negligent acts 2016*,Kidnapping and related offences 2003,Kidnapping and related offences 2004,Kidnapping and related offences 2005,Kidnapping and related offences 2006,Kidnapping and related offences 2007,Kidnapping and related offences 2008,Kidnapping and related offences 2009,Kidnapping and related offences 2010,Kidnapping and related offences 2011,Kidnapping and related offences 2012,Kidnapping and related offences 2013,Kidnapping and related offences 2014,Kidnapping and related offences 2015,Kidnapping and related offences 2016*,"Robbery, extortion and hijacking offences 2003","Robbery, extortion and hijacking offences 2004","Robbery, extortion and hijacking offences 2005","Robbery, extortion and hijacking offences 2006","Robbery, extortion and hijacking offences 2007","Robbery, extortion and hijacking offences 2008","Robbery, extortion and hijacking offences 2009","Robbery, extortion and hijacking offences 2010","Robbery, extortion and hijacking offences 2011","Robbery, extortion and hijacking offences 2012","Robbery, extortion and hijacking offences 2013","Robbery, extortion and hijacking offences 2014","Robbery, extortion and hijacking offences 2015","Robbery, extortion and hijacking offences 2016*",Burglary and related offences 2003,Burglary and related offences 2004,Burglary and related offences 2005,Burglary and related offences 2006,Burglary and related offences 2007,Burglary and related offences 2008,Burglary and related offences 2009,Burglary and related offences 2010,Burglary and related offences 2011,Burglary and related offences 2012,Burglary and related offences 2013,Burglary and related offences 2014,Burglary and related offences 2015,Burglary and related offences 2016*,Theft and related offences 2003,Theft and related offences 2004,Theft and related offences 2005,Theft and related offences 2006,Theft and related offences 2007,Theft and related offences 2008,Theft and related offences 2009,Theft and related offences 2010,Theft and related offences 2011,Theft and related offences 2012,Theft and related offences 2013,Theft and related offences 2014,Theft and related offences 2015,Theft and related offences 2016*,"Fraud, deception and related offences 2003","Fraud, deception and related offences 2004","Fraud, deception and related offences 2005","Fraud, deception and related offences 2006","Fraud, deception and related offences 2007","Fraud, deception and related offences 2008","Fraud, deception and related offences 20

In [14]:
dublin_crimes = crimes_df[crimes_df['Divisions'].str.contains("D.M.R")]
dublin_crimes.reset_index(inplace=True)
dublin_crimes

,index,id,Station,Divisions,x,y,"Attempts or threats to murder, assaults, harassments and related offences 2004","Attempts or threats to murder, assaults, harassments and related offences 2005","Attempts or threats to murder, assaults, harassments and related offences 2006","Attempts or threats to murder, assaults, harassments and related offences 2007","Attempts or threats to murder, assaults, harassments and related offences 2008","Attempts or threats to murder, assaults, harassments and related offences 2009","Attempts or threats to murder, assaults, harassments and related offences 2010","Attempts or threats to murder, assaults, harassments and related offences 2011","Attempts or threats to murder, assaults, harassments and related offences 2012","Attempts or threats to murder, assaults, harassments and related offences 2013","Attempts or threats to murder, assaults, harassments and related offences 2014","Attempts or threats to murder, assaults, harassments and related offences 2015","Attempts or threats to murder, assaults, harassments and related offences 2016*",Dangerous or negligent acts 2003,Dangerous or negligent acts 2004,Dangerous or negligent acts 2005,Dangerous or negligent acts 2006,Dangerous or negligent acts 2007,Dangerous or negligent acts 2008,Dangerous or negligent acts 2009,Dangerous or negligent acts 2010,Dangerous or negligent acts 2011,Dangerous or negligent acts 2012,Dangerous or negligent acts 2013,Dangerous or negligent acts 2014,Dangerous or negligent acts 2015,Dangerous or negligent acts 2016*,Kidnapping and related offences 2003,Kidnapping and related offences 2004,Kidnapping and related offences 2005,Kidnapping and related offences 2006,Kidnapping and related offences 2007,Kidnapping and related offences 2008,Kidnapping and related offences 2009,Kidnapping and related offences 2010,Kidnapping and related offences 2011,Kidnapping and related offences 2012,Kidnapping and related offences 2013,Kidnapping and related offences 2014,Kidnapping and related offences 2015,Kidnapping and related offences 2016*,"Robbery, extortion and hijacking offences 2003","Robbery, extortion and hijacking offences 2004","Robbery, extortion and hijacking offences 2005","Robbery, extortion and hijacking offences 2006","Robbery, extortion and hijacking offences 2007","Robbery, extortion and hijacking offences 2008","Robbery, extortion and hijacking offences 2009","Robbery, extortion and hijacking offences 2010","Robbery, extortion and hijacking offences 2011","Robbery, extortion and hijacking offences 2012","Robbery, extortion and hijacking offences 2013","Robbery, extortion and hijacking offences 2014","Robbery, extortion and hijacking offences 2015","Robbery, extortion and hijacking offences 2016*",Burglary and related offences 2003,Burglary and related offences 2004,Burglary and related offences 2005,Burglary and related offences 2006,Burglary and related offences 2007,Burglary and related offences 2008,Burglary and related offences 2009,Burglary and related offences 2010,Burglary and related offences 2011,Burglary and related offences 2012,Burglary and related offences 2013,Burglary and related offences 2014,Burglary and related offences 2015,Burglary and related offences 2016*,Theft and related offences 2003,Theft and related offences 2004,Theft and related offences 2005,Theft and related offences 2006,Theft and related offences 2007,Theft and related offences 2008,Theft and related offences 2009,Theft and related offences 2010,Theft and related offences 2011,Theft and related offences 2012,Theft and related offences 2013,Theft and related offences 2014,Theft and related offences 2015,Theft and related offences 2016*,"Fraud, deception and related offences 2003","Fraud, deception and related offences 2004","Fraud, deception and related offences 2005","Fraud, deception and related offences 2006","Fraud, deception and related offences 2007","Fraud, deception and related offences 2008","Fraud, deception and related offen

In [15]:
dublin_crimes['avg_crime_rate_since_2004'] = dublin_crimes.iloc[:,6:].mean(axis=1)
dublin_crimes = dublin_crimes[['Station', 'avg_crime_rate_since_2004']]
dublin_crimes

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Station,avg_crime_rate_since_2004
0,Balbriggan,94.185629
1,Ballyfermot,178.203593
2,Ballymun,159.155689
3,Blackrock Co Dublin,122.461078
4,Blanchardstown,446.137725
5,Bridewell Dublin,452.772455
6,Cabinteely,90.287425
7,Cabra,114.964072
8,Clondalkin,216.856287
9,Clontarf,149.670659


In [19]:
geolocator = Nominatim(user_agent="dublin_stations")

for station in dublin_crimes['Station']:
    location = geolocator.geocode(station)
    
    latitude = location.latitude
    longitude = location.longitude
    print('The geograpical coordinate of {} are {}, {}.'.format(station, latitude, longitude))
    
    dublin_crimes = dublin_crimes.loc[dublin_crimes['Station']==station]['Latitude'] = latitude
    dublin_crimes dublin_crimes.loc[dublin_crimes['Station']==station]['Longitude'] = longitude 
    
dublin_crimes.head()

The geograpical coordinate of Balbriggan are 53.609832, -6.1861964.


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


The geograpical coordinate of Ballyfermot are 53.3423203, -6.3488839.
The geograpical coordinate of Ballymun are 53.3948616, -6.2651936.
The geograpical coordinate of Blackrock Co Dublin are 53.3018639, -6.1788342.
The geograpical coordinate of Blanchardstown are 53.3868998, -6.3775408.
The geograpical coordinate of Bridewell Dublin are 53.3472221, -6.2741489.
The geograpical coordinate of Cabinteely are 53.2612843, -6.1505691.
The geograpical coordinate of Cabra are 37.4718153, -4.4335616.
The geograpical coordinate of Clondalkin are 53.3219624, -6.3942689.
The geograpical coordinate of Clontarf are 45.37635, -95.678651.
The geograpical coordinate of Coolock are 53.3892461, -6.2004097.
The geograpical coordinate of Crumlin are 51.6781517, -3.1404356.
The geograpical coordinate of Donnybrook are 48.509465, -101.8851618.
The geograpical coordinate of Dublin Airport are 53.4288026, -6.247592522077657.
The geograpical coordinate of Dun Laoghaire are 53.2922794, -6.1360079.
The geograpical

,Station,avg_crime_rate_since_2004
0,Balbriggan,94.185629
1,Ballyfermot,178.203593
2,Ballymun,159.155689
3,Blackrock Co Dublin,122.461078
4,Blanchardstown,446.137725


In [ ]:
gdf = geopandas.GeoDataFrame(
    dublin_crimes, geometry=geopandas.points_from_xy(df.Longitude, df.Latitude))

### 4) Foursquare Places API
Finally, the last part involves a similar approach taken during the previous weeks in this course where we had analysed different neighborhoods in Toronto, Canada.  
The challenge here is to obtain different districts comprising within Dublin City and obtain their respectice geographical coordinates using Nominatim geolocator.  
The sample code given below shows how we plan to construct the final dataframe where each row would be an individual venue along-with the attributes of each of the venues including their geolcation coordinates.  
OneHotEncoding can be used to get a feature representing distribution of different types of venues as well as the most popular and dominating venue type in each of the districts within Dublin city.  

In [6]:
CLIENT_ID = 'WV2XS4MH5YRWGHLTCFT4CKR4SRWNHWAF3JHWHNN4MKEQWTL3' # your Foursquare ID
CLIENT_SECRET = 'QEWWIOG0M3BT4V0YPNSKJY521MDUBHBYWBXCJFZ0452KP3OT' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
limit=100

In [77]:
def getNearbyVenues(names, latitudes, longitudes, radius=2000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [72]:
dublin_df = pd.read_csv('dublin_neighborhoods.csv', encoding = "utf-8")
dublin_df

,Neighborhood,Latitude,Longitude
0,Dublin 1,NaN,NaN
1,Dublin 2,NaN,NaN
2,Dublin 3,NaN,NaN
3,Dublin 4,NaN,NaN
4,Dublin 5,NaN,NaN
5,Dublin 6,NaN,NaN
6,Dublin 6W,NaN,NaN
7,Dublin 7,NaN,NaN
8,Dublin 8,NaN,NaN
9,Dublin 9,NaN,NaN


In [73]:
geolocator = Nominatim(user_agent="dublin_neighborhoods")

for district in dublin_df['Neighborhood']:
    location = geolocator.geocode(district)
    
    latitude = location.latitude
    longitude = location.longitude
    print('The geograpical coordinate of {} are {}, {}.'.format(district, latitude, longitude))
    
    dublin_df.loc[dublin_df['Neighborhood']==district, ['Latitude']] = latitude
    dublin_df.loc[dublin_df['Neighborhood']==district, ['Longitude']] = longitude 
    
dublin_df.head()

The geograpical coordinate of Dublin 1 are 53.3524881, -6.256645689721826.
The geograpical coordinate of Dublin 2 are 53.33894015, -6.252712821759609.
The geograpical coordinate of Dublin 3 are 53.361223100000004, -6.1854668060000355.
The geograpical coordinate of Dublin 4 are 53.32750729999999, -6.227485885927834.
The geograpical coordinate of Dublin 5 are 53.3834538, -6.181923245473566.
The geograpical coordinate of Dublin 6 are 53.3176976, -6.259525132569765.
The geograpical coordinate of Dublin 6W are 53.30928205, -6.299434891747282.
The geograpical coordinate of Dublin 7 are 53.3605505, -6.284470454564643.
The geograpical coordinate of Dublin 8 are 53.350262900000004, -6.320212883866121.
The geograpical coordinate of Dublin 9 are 53.3860497, -6.245577085317763.
The geograpical coordinate of Dublin 10 are 53.34321655, -6.360963597131269.
The geograpical coordinate of Dublin 11 are 53.386613600000004, -6.292626932293775.
The geograpical coordinate of Dublin 12 are 53.32052905, -6.32

,Neighborhood,Latitude,Longitude
0,Dublin 1,53.352488,-6.256646
1,Dublin 2,53.338940,-6.252713
2,Dublin 3,53.361223,-6.185467
3,Dublin 4,53.327507,-6.227486
4,Dublin 5,53.383454,-6.181923


In [78]:
dublin_venues = getNearbyVenues(names=dublin_df['Neighborhood'],
                                   latitudes=dublin_df['Latitude'],
                                   longitudes=dublin_df['Longitude']
                                  )

Dublin 1
Dublin 2
Dublin 3
Dublin 4
Dublin 5
Dublin 6
Dublin 6W
Dublin 7
Dublin 8
Dublin 9
Dublin 10
Dublin 11
Dublin 12
Dublin 13
Dublin 14
Dublin 15
Ballinteer
Dublin 17
Dublin 18
Dublin 20
Clondalkin
Tallaght
Blackrock
Dun Laoghaire
Swords


In [79]:
dublin_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Dublin 1,53.352488,-6.256646,147 Deli,53.353410,-6.259807,Deli / Bodega
1,Dublin 1,53.352488,-6.256646,The Celt,53.350442,-6.255071,Pub
2,Dublin 1,53.352488,-6.256646,Gate Theatre,53.353113,-6.261997,Theater
3,Dublin 1,53.352488,-6.256646,Murray's Bar,53.352419,-6.261256,Pub
4,Dublin 1,53.352488,-6.256646,Pull&Bear,53.349533,-6.262053,Clothing Store
5,Dublin 1,53.352488,-6.256646,Dealz,53.350623,-6.263183,Discount Store
6,Dublin 1,53.352488,-6.256646,Shoe Lane Coffee,53.347147,-6.255075,Café
7,Dublin 1,53.352488,-6.256646,Vice Coffee Inc.,53.347915,-6.262327,Coffee Shop
8,Dublin 1,53.352488,-6.256646,The Famine Memorial,53.348059,-6.250108,Sculpture Garden
9,Dublin 1,53.352488,-6.256646,Offbeat Donut Co,53.347435,-6.255535,Donut Shop


In [80]:
dublin_venues.groupby('Neighborhood').count()['Venue']

Neighborhood
Ballinteer        76
Blackrock         70
Clondalkin        36
Dublin 1         100
Dublin 10         29
Dublin 11         31
Dublin 12         39
Dublin 13         57
Dublin 14        100
Dublin 15         77
Dublin 17         44
Dublin 18         31
Dublin 2         100
Dublin 20         60
Dublin 3          44
Dublin 4         100
Dublin 5          47
Dublin 6         100
Dublin 6W         73
Dublin 7         100
Dublin 8          72
Dublin 9          59
Dun Laoghaire     75
Swords            72
Tallaght          54
Name: Venue, dtype: int64

In [81]:
address = 'Dublin, Ireland'

geolocator = Nominatim(user_agent="dublin_locator")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Dublin are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Dublin are 53.3497645, -6.2602732.


In [83]:
# create map of Dublin using latitude and longitude values
map_dublin = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(dublin_df['Latitude'], dublin_df['Longitude'], dublin_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_dublin)  
    
map_dublin

In [85]:
print('There are {} uniques categories.'.format(len(dublin_venues['Venue Category'].unique())))

There are 192 uniques categories.


In [86]:
# one hot encoding
dublin_onehot = pd.get_dummies(dublin_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
dublin_onehot['Neighborhood'] = dublin_venues['Neighborhood'] 

# move neighborhood column to the first column
cols = list(dublin_onehot)
cols.insert(0, cols.pop(cols.index('Neighborhood')))
dublin_onehot = dublin_onehot.loc[:, cols]

dublin_onehot.head()

,Neighborhood,American Restaurant,Arcade,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Betting Shop,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Bowling Alley,Breakfast Spot,Brewery,Bridge,Buffet,Burger Joint,Burrito Place,Bus Stop,Café,Canal,Canal Lock,Castle,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comic Shop,Concert Hall,Convenience Store,Convention Center,Creperie,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Donut Shop,Electronics Store,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Health Food Store,Historic Site,History Museum,Hockey Arena,Hockey Field,Home Service,Hotel,Hotel Bar,Hungarian Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Irish Pub,Island,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Light Rail Station,Lighthouse,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multiplex,Museum,Music Venue,Nail Salon,Nature Preserve,Optical Shop,Other Great Outdoors,Outdoor Sculpture,Outlet Store,Paper / Office Supplies Store,Park,Pedestrian Plaza,Pet Store,Pharmacy,Pier,Pizza Place,Platform,Playground,Plaza,Pool,Pool Hall,Port,Portuguese Restaurant,Pub,Recreation Center,Rental Car Location,Restaurant,Rugby Pitch,Salad Place,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Soccer Field,Soccer Stadium,Souvenir Shop,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Surf Spot,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Tourist Information Center,Toy / Game Store,Track,Trail,Train Station,Tram Station,Tunnel,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,Dublin 1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Dublin 1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Dublin 1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Dublin 1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [87]:
dublin_onehot.shape

(1646, 193)

In [89]:
dublin_grouped = dublin_onehot.groupby('Neighborhood').mean().reset_index()
dublin_grouped

,Neighborhood,American Restaurant,Arcade,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Betting Shop,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Bowling Alley,Breakfast Spot,Brewery,Bridge,Buffet,Burger Joint,Burrito Place,Bus Stop,Café,Canal,Canal Lock,Castle,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comic Shop,Concert Hall,Convenience Store,Convention Center,Creperie,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Donut Shop,Electronics Store,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Health Food Store,Historic Site,History Museum,Hockey Arena,Hockey Field,Home Service,Hotel,Hotel Bar,Hungarian Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Irish Pub,Island,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Light Rail Station,Lighthouse,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multiplex,Museum,Music Venue,Nail Salon,Nature Preserve,Optical Shop,Other Great Outdoors,Outdoor Sculpture,Outlet Store,Paper / Office Supplies Store,Park,Pedestrian Plaza,Pet Store,Pharmacy,Pier,Pizza Place,Platform,Playground,Plaza,Pool,Pool Hall,Port,Portuguese Restaurant,Pub,Recreation Center,Rental Car Location,Restaurant,Rugby Pitch,Salad Place,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Soccer Field,Soccer Stadium,Souvenir Shop,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Surf Spot,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Tourist Information Center,Toy / Game Store,Track,Trail,Train Station,Tram Station,Tunnel,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,Ballinteer,0.000000,0.00,0.000000,0.000000,0.013158,0.000000,0.000000,0.00,0.000000,0.013158,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.026316,0.00,0.000000,0.013158,0.000000,0.00,0.013158,0.013158,0.000000,0.000000,0.065789,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.052632,0.000000,0.052632,0.013158,0.00,0.013158,0.00,0.00,0.000000,0.039474,0.00,0.000000,0.026316,0.00,0.000000,0.013158,0.000000,0.013158,0.013158,0.013158,0.000000,0.000000,0.000000,0.000000,0.013158,0.000000,0.013158,0.00,0.026316,0.000000,0.000000,0.000000,0.013158,0.00,0.000000,0.013158,0.000000,0.000000,0.000000,0.039474,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.013158,0.026316,0.00,0.000000,0.00,0.000000,0.026316,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.013158,0.000000,0.000000,0.000000,0.000000,0.000000,0.013158,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.026316,0.00,0.013158,0.013158,0.000000,0.026316,0.000000,0.013158,0.00,0.013158,0.000000,0.000000,0.013158,0.052632,0.00,0.000000,0.000000,0.000000,0.000000,0.013158,0.000000,0.000000,0.000000,0.000000,0.000000,0.026316,0.013158,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.013158,0.00,0.000000,0.105263,0.000000,0.000000,0.00,0.013158,0.000000,0.0

In [90]:
dublin_grouped.shape

(25, 193)

In [91]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [95]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = dublin_grouped['Neighborhood']

for ind in np.arange(dublin_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(dublin_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Ballinteer,Supermarket,Café,Pub,Coffee Shop,Clothing Store,Department Store,Gym,Furniture / Home Store,Italian Restaurant,Park
1,Blackrock,Pub,Café,Train Station,Park,Coffee Shop,Shopping Mall,Supermarket,Bar,Thai Restaurant,Italian Restaurant
2,Clondalkin,Hotel,Bar,Convenience Store,Coffee Shop,Restaurant,Supermarket,Chinese Restaurant,Light Rail Station,Gym,Golf Course
3,Dublin 1,Coffee Shop,Café,Pub,Park,Italian Restaurant,Hotel,Bookstore,Theater,Bar,Plaza
4,Dublin 10,Supermarket,Pub,Park,Gym,Hotel,Coffee Shop,Fast Food Restaurant,Hardware Store,Bowling Alley,Café
5,Dublin 11,Supermarket,Park,Convenience Store,Pub,Grocery Store,Sandwich Place,Tram Station,Sporting Goods Shop,Breakfast Spot,Chinese Restaurant
6,Dublin 12,Supermarket,Park,Convenience Store,Fast Food Restaurant,Tram Station,Coffee Shop,Grocery Store,Hardware Store,Shopping Mall,Motorcycle Shop
7,Dublin 13,Seafood Restaurant,Pub,Café,Fish Market,Ice Cream Shop,Harbor / Marina,Golf Course,Coffee Shop,Bar,Breakfast Spot
8,Dublin 14,Supermarket,Pub,Coffee Shop,Clothing Store,Café,Department Store,Pizza Place,Restaurant,Discount Store,Pharmacy
9,Dublin 15,Coffee Shop,Supermarket,Clothing Store,Italian Restaurant,Train Station,Fast Food Restaurant,Furniture / Home Store,Pub,Asian Restaurant,Sporting Goods Shop


In [101]:
# set number of clusters
kclusters = 3

dublin_grouped_clustering = dublin_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dublin_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([1, 1, 1, 0, 2, 2, 2, 0, 1, 1, 2, 1, 0, 1, 1, 0, 2, 0, 2, 0, 1, 2,
       0, 1, 2], dtype=int32)

In [100]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

dublin_merged = dublin_df

# merge dublin_grouped with dublin_merged to add latitude/longitude for each neighborhood
dublin_merged = dublin_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
dublin_merged['Cluster Labels'].fillna(3.0, inplace=True)
dublin_merged['Cluster Labels'] = dublin_merged['Cluster Labels'].astype(int)

dublin_merged.head() # check the last columns!

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Dublin 1,53.352488,-6.256646,0,Coffee Shop,Café,Pub,Park,Italian Restaurant,Hotel,Bookstore,Theater,Bar,Plaza
1,Dublin 2,53.338940,-6.252713,0,Café,Coffee Shop,Park,Hotel,Plaza,Pub,Restaurant,Cocktail Bar,Grocery Store,Bakery
2,Dublin 3,53.361223,-6.185467,1,Café,Pub,Boat or Ferry,Beach,Park,Scenic Lookout,Convenience Store,Train Station,Restaurant,Port
3,Dublin 4,53.327507,-6.227486,0,Pub,Café,Restaurant,Coffee Shop,Hotel,Park,Grocery Store,Gastropub,Pizza Place,Plaza
4,Dublin 5,53.383454,-6.181923,2,Supermarket,Grocery Store,Train Station,Convenience Store,Fast Food Restaurant,Pub,Shopping Mall,Café,Pizza Place,Bus Stop


In [131]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10.2)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dublin_merged['Latitude'], dublin_merged['Longitude'], dublin_merged['Neighborhood'], dublin_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=6,
        popup=label,
        color=rainbow[cluster],
        fill=True,
        fill_color=rainbow[cluster],
        fill_opacity=0.3).add_to(map_clusters)
       
map_clusters